## Sentinel 2 Analysis Using Google Earth Engine

cloud score and shadow TDOM methods adpated for Python by me from Javascript by Ian Housman:
https://groups.google.com/forum/#!msg/google-earth-engine-developers/i63DS-Dg8Sg/FGuT5OtSAQAJ;context-place=msg/google-earth-engine-developers/jYFuopAlIi0/tf2AKG4_BAAJ
 

In [ ]:
import ee
ee.Initialize()
import matplotlib
from matplotlib.pylab import plt
import numpy as np
import pandas as pd
import seaborn as sns
#print(s2c.getInfo())
%matplotlib inline



In [ ]:
#S2 cloud score and shadow removal parameters
cloudThresh =10 #lower value more cloud filtered
irSumThresh =0.35 #Sum of IR bands to include as shadows within TDOM and the shadow shift method (lower number masks out less)
dilatePixels = 2 #Pixels to dilate around clouds
contractPixels = 1 #Pixels to reduce cloud mask and dark shadows by to reduce inclusion of single-pixel comission errors

In [ ]:
#Bands are divided by 10000 and renamed for cloud score
def s2_bands(img):
    t = img.select([ 'B1','B2','B3','B4','B5','B6','B7','B8','B8A', 'B9','B10', 'B11','B12']).divide(10000)
    t = t.addBands(img.select(['QA60']))
    out = t.copyProperties(img).copyProperties(img,['system:time_start'])
    return out

In [ ]:
#Masking function for S2 cloud using QA band
def maskS2clouds(image):
    qa = image.select('QA60')
     #Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 2**10
    cirrusBitMask = 2**11
    #Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0)
    #.and(qa.bitwiseAnd(cirrusBitMask).eq(0));
    return image.updateMask(mask)

In [ ]:
def rescale(img, exp, thresholds):
    return img.expression(exp,{'img':img}).subtract(thresholds[0]).divide(thresholds[1] - thresholds[0])

In [ ]:
#Masking function for S2 cloud using cloud score 
def sentinelCloudScore(img):
  # Compute several indicators of cloudyness and take the minimum of them.
    score = ee.Image(1)  
  # Clouds are reasonably bright in the blue and cirrus bands.
    score = score.min(rescale(img, 'img.blue', [0.1, 0.5]))
    score = score.min(rescale(img, 'img.cb', [0.1, 0.3]))
    score = score.min(rescale(img, 'img.cb + img.cirrus', [0.15, 0.2])) 
  # Clouds are reasonably bright in all visible bands.
    score = score.min(rescale(img, 'img.red + img.green + img.blue', [0.2, 0.8]))
  # Clouds are moist
    ndmi = img.normalizedDifference(['nir','swir1'])
    score=score.min(rescale(ndmi, 'img', [-0.1, 0.1]))
  # However, clouds are not snow.
    ndsi = img.normalizedDifference(['green', 'swir1'])
    score=score.min(rescale(ndsi, 'img', [0.8, 0.6]))
    score = score.multiply(100).byte()
    return img.addBands(score.rename('cloudScore'))

In [ ]:
#Function to remove clouds from S2 image using cloud score
def scoreRemoveClouds(img):
    img = sentinelCloudScore(img)
    img = img.updateMask(img.select(['cloudScore']).gt(cloudThresh).focal_min(contractPixels).focal_max(dilatePixels).Not())
    return img

In [ ]:
# TDOM2 function for removing cloud shadow
def simpleTDOM2(coll):
    shadowSumBands = ['nir','swir1']
    irSumThresh = 0.4
    zShadowThresh = -1.2
    #Get some pixel-wise stats for the time series
    irStdDev = coll.select(shadowSumBands).reduce(ee.Reducer.stdDev())
    irMean = coll.select(shadowSumBands).mean()
    bandNames = ee.Image(coll.first()).bandNames()
    #Mask out dark outliers using mask dark function
    def mask_dark(img):
        z = img.select(shadowSumBands).subtract(irMean).divide(irStdDev)
        irSum = img.select(shadowSumBands).reduce(ee.Reducer.sum())
        m = z.lt(zShadowThresh).reduce(ee.Reducer.sum()).eq(2).And(irSum.lt(irSumThresh)).Not()
        return img.updateMask(img.mask().And(m))
    coll = coll.map(mask_dark)
    return coll.select(bandNames)

In [ ]:
#Test s2 cloud shadow masks on one image
n = s2c.size()
colList = s2c.toList(n)
img1 = maskS2clouds(ee.Image(colList.get(0)))
img1 = sentinelCloudScore(img1)
img1 = img1.updateMask(img1.select(['cloudScore']).gt(cloudThresh).focal_min(contractPixels).focal_max(dilatePixels).Not())

val = img1.reduceRegions(pnts,ee.Reducer.first(),10)

In [ ]:
#Get S2 collection
s2c = ee.ImageCollection('COPERNICUS/S2')\
.filterDate('2018-09-01', '2018-09-30')\
.filterBounds(pnts)\
.sort('CLOUDY_PIXEL_PERCENTAGE')\
.map(s2_bands)\
.select(['QA60', 'B1','B2','B3','B4','B5','B6','B7','B8','B8A', 'B9','B10', 'B11','B12'],['QA60','cb', 'blue', 'green', 'red', 're1','re2','re3','nir', 'nir2', 'waterVapor', 'cirrus','swir1', 'swir2'])\
.map(scoreRemoveClouds)

#Apply TDOM
#s2c = simpleTDOM2(s2c)
s2img = s2c.first()

In [ ]:
val = s2img.reduceRegions(pnts,ee.Reducer.first(),10)

#task=ee.batch.Export.table.toDrive(collection=val,folder = 'Pixel_Values',fileFormat='CSV',fileNamePrefix='fnfssd')
t=ee.batch.Export.image.toAsset(image=s2img, description='imageToAssetEx',assetId='users/tomw_ee/testme', scale=10)
t.start()